In [19]:
# Now I want write a test script to test all testcases in testcases folder
# The testcases in such format
# <name>.sy source code
# <name>.in input
# <name>.out std output

# compiler path: ./build/SYSY-compiler
# compiler command: ./build/SYSY-compiler <name>.sy
# I need you to run the compiler and get the output llvm ir code, which will be put in <name>.ll 
# If the compiler doesn't end with 0, keep a record of the trouble file name in CE_list
# If the compiler ends with 0, run the llvm ir code with lli and get the output, which will be put in <name>.outq
# Compare it with the std output <name>.out ,if is't not that same ,keep a record of the trouble file name in WA_list
# Remember to redirect the input and output of lli

import os
import sys
import subprocess
import filecmp
import difflib

compiler_path = "./build/SYSY-compiler"
test_folder="./testcases"

pass_args=["--mem2reg","--constprop","--dce"]

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".sy"):
            test_list.append(os.path.join(root, file))

CE_list = []
LLI_FAIL_list = []
WA_list = []
AC_list = []
TLE_list = []

Bad_test = []

for test in test_list:
    if test.endswith(".sy"):
        # add test_args to list
        compile_args=[compiler_path, test]
        for arg in pass_args:
            compile_args.append(arg)
        ret = subprocess.run(compile_args)
        if ret.returncode != 0:
            CE_list.append(test)
            # print("Compiler Error")
            continue
        # run lli
        ll_file = test+".ll"
        out_file = test[:-2]+"out"
        # maybe theres no input file
        
        if not os.path.exists(test[:-2]+"in") and not os.path.exists(out_file):
            Bad_test.append(test)
            continue
        
        # Kill program run over 5s, give it a TLE
        try:
            if not os.path.exists(test[:-2]+"in"):
                ret = subprocess.run(["lli","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=5)
            else:
                ret = subprocess.run(["lli","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=5)
        except subprocess.TimeoutExpired:
            # print("Timeout Error")
            TLE_list.append(test)
            continue
            
        # compare the output
        # Merge The reture Value and stdout
        
        dump_str=ret.stdout.decode()
        # remove whitesspace in the end
        # dump_str=dump_str.rstrip()
        if dump_str and not dump_str.endswith('\n'):
            dump_str += "\n"
        dump_str += str(ret.returncode) + "\n"
        std_output=open(out_file).read()
        diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
        if(len(list(diff))!=0):
            # print("Wrong Answer")
            WA_list.append(test)
            continue
        # print("Accepted")
        AC_list.append(test)
        
print("Compiler Error: Total: "+str(len(CE_list)))
print("Runtime Error: Total: "+str(len(LLI_FAIL_list)))
print("Timeout Error: Total: "+str(len(TLE_list)))
print("Wrong Answer: Total: "+str(len(WA_list)))
print("Bad Test: Total: "+str(len(Bad_test)))
print("Accepted: Total: "+str(len(AC_list)))

Compiler Error: Total: 0
Runtime Error: Total: 0
Timeout Error: Total: 1
Wrong Answer: Total: 86
Bad Test: Total: 62
Accepted: Total: 61


In [20]:
for k in range(len(WA_list)):
    if len(WA_list) > k:
        print(WA_list[k])


./testcases/80_chaos_token.sy
./testcases/25_scope3.sy
./testcases/85_long_code.sy
./testcases/83_long_array.sy
./testcases/71_full_conn.sy
./testcases/51_short_circuit3.sy
./testcases/59_sort_test5.sy
./testcases/96_matrix_add.sy
./testcases/19_search.sy
./testcases/issue-115.sy
./testcases/76_n_queens.sy
./testcases/28_side_effect2.sy
./testcases/issue-121.sy
./testcases/84_long_array2.sy
./testcases/72_hanoi.sy
./testcases/74_kmp.sy
./testcases/99_matrix_tran.sy
./testcases/52_scope.sy
./testcases/67_reverse_output.sy
./testcases/48_assign_complex_expr.sy
./testcases/issue-125.sy
./testcases/82_long_func.sy
./testcases/17_maximal_clique.sy
./testcases/39_op_priority5.sy
./testcases/98_matrix_mul.sy
./testcases/issue-131.sy
./testcases/15_graph_coloring.sy
./testcases/05_arr_defn4.sy
./testcases/36_rotate.sy
./testcases/58_sort_test4.sy
./testcases/87_many_params.sy
./testcases/25_while_if.sy
./testcases/26_scope4.sy
./testcases/91_many_locals2.sy
./testcases/88_many_params2.sy
./tes

In [15]:
for k in range(len(TLE_list)):
    if len(TLE_list) > k:
        print(TLE_list[k])


./testcases/24_array_only.sy


In [16]:
for k in range(len(CE_list)):
    if len(CE_list) > k:
        print(CE_list[k])
